In [2]:
import numpy as np
import pandas as pd

In [28]:
crop_reco=pd.read_csv("datasets/crop_recommendation.csv")
crop_reco

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice
...,...,...,...,...,...,...,...,...
2195,107,34,32,26.774637,66.413269,6.780064,177.774507,coffee
2196,99,15,27,27.417112,56.636362,6.086922,127.924610,coffee
2197,118,33,30,24.131797,67.225123,6.362608,173.322839,coffee
2198,117,32,34,26.272418,52.127394,6.758793,127.175293,coffee


In [4]:
crop_prices=pd.read_csv("datasets/crop_prices.csv")
crop_prices.head()

,state,district,market,commodity,variety,arrival_date,min_price,max_price,modal_price
0,Andaman and Nicobar,South Andaman,Port Blair,Amaranthus,Amaranthus,04/03/2019,6000,8000,7000
1,Andaman and Nicobar,South Andaman,Port Blair,Banana - Green,Banana - Green,04/03/2019,4500,5500,5000
2,Andaman and Nicobar,South Andaman,Port Blair,Bhindi(Ladies Finger),Bhindi,04/03/2019,6000,8000,7000
3,Andaman and Nicobar,South Andaman,Port Blair,Bitter gourd,Other,04/03/2019,6000,8000,7000
4,Andaman and Nicobar,South Andaman,Port Blair,Black pepper,Other,04/03/2019,110000,130000,120000


In [5]:
#dropping and renaming
crop_prices.drop(['market', 'variety', 'arrival_date', 'min_price', 'max_price'], axis=1, inplace=True)
crop_prices.rename(columns = {'modal_price':'price', 'commodity':'crop'}, inplace = True)
crop_prices.head()

,state,district,crop,price
0,Andaman and Nicobar,South Andaman,Amaranthus,7000
1,Andaman and Nicobar,South Andaman,Banana - Green,5000
2,Andaman and Nicobar,South Andaman,Bhindi(Ladies Finger),7000
3,Andaman and Nicobar,South Andaman,Bitter gourd,7000
4,Andaman and Nicobar,South Andaman,Black pepper,120000


In [6]:
district_rain=pd.read_csv("datasets/district_wise_rainfall.csv")
district_rain.head()

,STATE_UT_NAME,DISTRICT,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANNUAL,Jan-Feb,Mar-May,Jun-Sep,Oct-Dec
0,ANDAMAN And NICOBAR ISLANDS,NICOBAR,107.3,57.9,65.2,117.0,358.5,295.5,285.0,271.9,354.8,326.0,315.2,250.9,2805.2,165.2,540.7,1207.2,892.1
1,ANDAMAN And NICOBAR ISLANDS,SOUTH ANDAMAN,43.7,26.0,18.6,90.5,374.4,457.2,421.3,423.1,455.6,301.2,275.8,128.3,3015.7,69.7,483.5,1757.2,705.3
2,ANDAMAN And NICOBAR ISLANDS,N & M ANDAMAN,32.7,15.9,8.6,53.4,343.6,503.3,465.4,460.9,454.8,276.1,198.6,100.0,2913.3,48.6,405.6,1884.4,574.7
3,ARUNACHAL PRADESH,LOHIT,42.2,80.8,176.4,358.5,306.4,447.0,660.1,427.8,313.6,167.1,34.1,29.8,3043.8,123.0,841.3,1848.5,231.0
4,ARUNACHAL PRADESH,EAST SIANG,33.3,79.5,105.9,216.5,323.0,738.3,990.9,711.2,568.0,206.9,29.5,31.7,4034.7,112.8,645.4,3008.4,268.1


In [7]:
#dropping and renaming
district_rain.drop(district_rain.iloc[:, 2:14], axis=1, inplace=True)
district_rain.rename(columns = {'STATE_UT_NAME':'state', 'DISTRICT':'district', 'ANNUAL':'annual'}, inplace = True)
district_rain['annual_avg']=district_rain['annual']/12
district_rain.head()

,state,district,annual,Jan-Feb,Mar-May,Jun-Sep,Oct-Dec,annual_avg
0,ANDAMAN And NICOBAR ISLANDS,NICOBAR,2805.2,165.2,540.7,1207.2,892.1,233.766667
1,ANDAMAN And NICOBAR ISLANDS,SOUTH ANDAMAN,3015.7,69.7,483.5,1757.2,705.3,251.308333
2,ANDAMAN And NICOBAR ISLANDS,N & M ANDAMAN,2913.3,48.6,405.6,1884.4,574.7,242.775000
3,ARUNACHAL PRADESH,LOHIT,3043.8,123.0,841.3,1848.5,231.0,253.650000
4,ARUNACHAL PRADESH,EAST SIANG,4034.7,112.8,645.4,3008.4,268.1,336.225000


In [8]:
#making all state and district names lowercase
district_rain['state'] = district_rain['state'].str.lower()
crop_prices['state'] = crop_prices['state'].str.lower()
district_rain['district'] = district_rain['district'].str.lower()
crop_prices['district'] = crop_prices['district'].str.lower()

In [9]:
#removing the states in rainfall dataset which dont exist in crop prices dataset
district_rain_temp = district_rain[district_rain['state'].isin(crop_prices['state'])]
x=list(district_rain_temp['state'].unique())

In [10]:
#finding out which states are named differently
s=set()
for i in crop_prices['state'].to_list():
    if i not in x:
        s.add(i)
print(s)

{'telangana', 'odisha', 'himachal pradesh', 'chattisgarh', 'uttrakhand', 'andaman and nicobar'}


In [11]:
#manually changing them
district_rain.replace({'uttaranchal':'uttrakhand', 'orissa':'odisha', 'chatisgarh':'chattisgarh', 'andaman and nicobar islands': 'andaman and nicobar', 'himachal':'himachal pradesh'}, 
                        regex=True, inplace=True)

In [12]:
#removing the states in rainfall dataset which dont exist in crop prices dataset again
district_rain = district_rain[district_rain['state'].isin(crop_prices['state'])]
x=list(district_rain['state'].unique())
print(len(x))

25


Special case - telangana

In [13]:
#districts in unsplit andhra pradesh 
both_districsts=list(district_rain[district_rain['state'] == 'andhra pradesh']['district'].unique())
print(both_districsts)

['east godavari', 'west godavari', 'guntur', 'krishna', 'nellore', 'prakasam', 'srikakulam', 'visakhapatnam', 'vizianagaram', 'adilabad', 'hyderabad', 'karimnagar', 'khammam', 'mahabubnagar', 'medak', 'nalgonda', 'nizamabad', 'warangal', 'rangareddy', 'anantapur', 'chittoor', 'kuddapah', 'kurnool']


In [14]:
#districts which come under telangana
telangana_districts=list(crop_prices[crop_prices['state']=='telangana']['district'].unique())
print(telangana_districts)

['hyderabad', 'karimnagar', 'khammam', 'nalgonda', 'nizamabad', 'ranga reddy']


In [15]:
#change andhra pradesh to telangana if district belongs to telangana
district_rain['state'] = np.where(district_rain['district'].isin(telangana_districts), 'telangana', district_rain['state'])
len(list(district_rain['state'].unique()))

26

In [24]:
crop_prices.groupby('state').apply(lambda x: len(x['district'].unique().tolist())).sort_index()

state
andaman and nicobar     1
andhra pradesh          3
assam                   6
chattisgarh             3
goa                     1
gujarat                 9
haryana                 6
himachal pradesh        2
jammu and kashmir       1
karnataka               3
kerala                  9
madhya pradesh          8
maharashtra            10
manipur                 5
meghalaya               1
nagaland                1
odisha                 10
pondicherry             1
punjab                 12
rajasthan               9
tamil nadu             18
telangana               6
tripura                 3
uttar pradesh          32
uttrakhand              2
west bengal            12
dtype: int64

In [26]:
district_rain_temp.groupby('state').apply(lambda x: len(x['district'].unique().tolist())).sort_index()

state
andhra pradesh       23
assam                27
goa                   2
gujarat              26
haryana              21
jammu and kashmir    22
karnataka            30
kerala               14
madhya pradesh       50
maharashtra          35
manipur               9
meghalaya             7
nagaland             11
pondicherry           4
punjab               20
rajasthan            33
tamil nadu           32
tripura               4
uttar pradesh        71
west bengal          19
dtype: int64

In [18]:
#seeing standard deviation of prices between districts of a state, to see if we can just take average for a state
print(crop_prices[crop_prices['state']=='manipur'].groupby('crop').std())
print(crop_prices[crop_prices['state']=='manipur'].groupby('crop').mean())

                                   price
crop                                    
Banana                        223.606798
Bengal Gram Dal (Chana Dal)  1501.110700
Cabbage                       250.998008
Cauliflower                   273.861279
Dry Chillies                  830.160627
Ginger(Green)                1140.175425
Masur Dal                     443.846820
Onion                         223.606798
Paddy(Dhan)(Common)           119.678388
Potato                        248.997992
Rice                          223.606798
Turmeric (raw)                       NaN
                                    price
crop                                     
Banana                        3400.000000
Bengal Gram Dal (Chana Dal)   9533.333333
Cabbage                       1260.000000
Cauliflower                   2200.000000
Dry Chillies                 14325.000000
Ginger(Green)                 4400.000000
Masur Dal                     7480.000000
Onion                         2400.000000
Paddy(

Exploring Crop types in datasets

In [27]:
crops_available=list(crop_reco['label'].unique())
print(crops_available)

['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas', 'mothbeans', 'mungbean', 'blackgram', 'lentil', 'pomegranate', 'banana', 'mango', 'grapes', 'watermelon', 'muskmelon', 'apple', 'orange', 'papaya', 'coconut', 'cotton', 'jute', 'coffee']


In [20]:
crop_prices['crop'].unique()

array(['Amaranthus', 'Banana - Green', 'Bhindi(Ladies Finger)',
       'Bitter gourd', 'Black pepper', 'Bottle gourd', 'Brinjal',
       'Cabbage', 'Carrot', 'Cauliflower', 'Cluster beans', 'Coconut',
       'Colacasia', 'Onion', 'Potato', 'Tomato',
       'Bengal Gram(Gram)(Whole)', 'Jowar(Sorghum)',
       'Paddy(Dhan)(Common)', 'Lentil (Masur)(Whole)', 'Rice',
       'Cucumbar(Kheera)', 'Field Pea', 'French Beans (Frasbean)',
       'Green Chilli', 'Knool Khol', 'Pumpkin', 'Raddish',
       'Black Gram (Urd Beans)(Whole)', 'Green Gram (Moong)(Whole)',
       'Jute', 'Maida Atta', 'Mustard', 'Wheat Atta', 'Garlic',
       'Masur Dal', 'Ridgeguard(Tori)', 'Arecanut(Betelnut/Supari)',
       'Arhar (Tur/Red Gram)(Whole)', 'Maize', 'Dry Chillies',
       'Groundnut', 'Capsicum', 'Guar', 'Lemon',
       'Bajra(Pearl Millet/Cumbu)', 'Castor Seed', 'Coriander(Leaves)',
       'Cowpea(Veg)', 'Drumstick', 'Elephant Yam (Suran)',
       'Ginger(Green)', 'Indian Beans (Seam)', 'Methi(Leaves)',

In [21]:
#change the name of crop to one of crop reco ones via regex
for i in crops_available:
    crop_prices['crop'] = crop_prices['crop'].str.replace(i, i, regex=True)

In [22]:
len(crop_prices['crop'].unique())

156

In [23]:
#correlation between humidity and rainfall
crop_reco['humidity'].corr(crop_reco['rainfall'])
#not much

0.0944230533485546

In [31]:
#the %sd between variables for each crop - checking to see if it can be reduced to 1 row
for i in crops_available:
    print(crop_reco[crop_reco['label']==i].std()/crop_reco[crop_reco['label']=='rice'].mean()*100)
    print()

N              14.917989
P              16.614052
K               7.389433
temperature     8.574627
humidity        1.723997
ph             11.963201
rainfall       14.519292
dtype: float64

N              14.957429
P              16.835852
K               7.377728
temperature    11.310406
humidity        6.622720
ph              6.422584
rainfall        6.582804
dtype: float64

N              15.209225
P              15.759868
K               8.181343
temperature     4.938247
humidity        2.080856
ph             12.467986
rainfall        3.363669
dtype: float64

N              13.561479
P              15.912367
K               7.780825
temperature    10.973600
humidity        2.634313
ph              2.260063
rainfall       11.055661
dtype: float64

N              14.832833
P              15.330944
K               7.060859
temperature    24.127662
humidity       13.325979
ph             12.921605
rainfall       13.966536
dtype: float64

N              14.198796
P              15.86

/var/folders/9p/s20qcdt565j49qt2_m5b682m0000gn/T/ipykernel_80297/275965326.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(crop_reco[crop_reco['label']==i].std()/crop_reco[crop_reco['label']=='rice'].mean()*100)
